In [ ]:
#import necessary modules
import numpy as np
import rasterio as rs
import matplotlib.pyplot as plt
import earthpy.spatial as es
import earthpy.plot as ep
from numpy import loadtxt
from osgeo import gdal, gdalconst
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from sklearn.ensemble import IsolationForest
from sklearn import svm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
#Set imagery as variables
asi = #'Path_to_SingleDateImage_Amsterdam.tif'
ami = #'Path_to_MedianImage_Amsterdam.tif'
msi = #'Path_to_SingleDateImage_Milano.tif'
mmi = #'Path_to_MedianImage_Milano.tif'
bsi = #'Path_to_SingleDateImage_Budapest.tif'
bmi = #'Path_to_MedianImage_Budapest.tif'
filist = [asi, ami, msi, mmi, bsi, bmi]

In [ ]:
#import Twitter Raster, Validation datasets & Anomaly score maps
Twit = rs.open(#'Path_to_TwitterRaster.tif','r').read()
Twit = np.transpose(Twit, [1, 2, 0])
Twit = Twit[:,:,0]
Val = rs.open(#'Path_to_30Mvalidationdataset.tif','r').read()
Val = np.transpose(Vala, [1, 2, 0])
Val30m = Val[:,:,0]
Val30m = np.where(Val30m==2, 1, 0)
Val30m = Val30m.flatten()
Val2 = rs.open(#'Path_to_GAIAvalidationdataset.tif','r').read()
Val2 = np.transpose(Val2, [1, 2, 0])
Val2GAIA = Val2[:,:,0]
Val2GAIA = np.where(Val2GAIA>0, 1, 0)
Val2GAIA = Val2GAIA.flatten()
Val3 = rs.open(#'Path_to_GHSvalidationdataset.tif','r').read()
Val3 = np.transpose(Val3, [1, 2, 0])
Val3GHS = Val3[:,:,0]
Val3GHS50 = np.where(Val3GHS>50, 1, 0)
Val3GHS50 = Val3GHS50.flatten()

AnomalyScore = np.loadtxt(#'Path_to_IsolationForest_img.csv', class_map1, delimiter=',')

In [ ]:
class_map = AnomalyScore
class_map = np.where(class_map>0.5, 1, 0) #first float is the Binary threshold
class_map = class_map.flatten()
pred = class_map
true1 = Val30m
true2 = Val2GAIA
true3 = Val3GHS50

cfm1 = confusion_matrix(true1, pred)
TN1,FP1,FN1,TP1= cfm1.ravel()
cfm2 = confusion_matrix(true2, pred)
TN2,FP2,FN2,TP2= cfm2.ravel()
cfm3 = confusion_matrix(true3, pred)
TN3,FP3,FN3,TP3= cfm3.ravel()

AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
      ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
      ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3

P = ((TP1/(TP1+FP1))+
     (TP2/(TP2+FP2))+
     (TP3/(TP3+FP3)))/3

R = ((TP1/(TP1+FN1))+
     (TP2/(TP2+FN2))+
     (TP3/(TP3+FN3)))/3

F = (((2*TP1)/(2*TP1+FP1+FN1))+
     ((2*TP2)/(2*TP2+FP2+FN2))+
     ((2*TP3)/(2*TP3+FP3+FN3)))/3

auc1 = roc_auc_score(true1,pred)
auc2 = roc_auc_score(true2,pred)
auc3 = roc_auc_score(true3,pred)
score = ((auc1+auc2+auc3)/3)

print('AUC score = ' + str(score))
print('Overall Accuracy = ' + str(AO))
print('Precision = ' + str(P))
print('Recall = ' + str(R))
print('F_score = ' + str(F))

In [ ]:
class_map = AnomalyScore
class_map = np.where(class_map>0.5, 1, 0)
fig, ax = plt.subplots(figsize=(12, 12))
im = ax.imshow(class_map, cmap="gray")